<a href="https://colab.research.google.com/github/vornitier/scraping/blob/main/scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Scraping text and use of OCR.

In [1]:
# Install the required packages
!apt update
!apt install chromium-chromedriver tesseract-ocr
!pip install selenium pytesseract webdriver-manager
!pip install --upgrade webdriver-manager

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
80 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
t

In [2]:
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

cp: '/usr/lib/chromium-browser/chromedriver' and '/usr/bin/chromedriver' are the same file


In [3]:
# Import necessary libraries
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import WebDriverException
from PIL import Image

import os
import pytesseract
import re
import time
import hashlib

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

# Setup Chrome options
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# Initialize the driver
driver = webdriver.Chrome(options=chrome_options)


In [4]:
# Function to sanitize website names into valid filenames
def sanitize_filename(url):
    # Remove protocol (http, https) and replace special characters with underscores
    sanitized_name = re.sub(r'https?://|www\.|/', '', url)
    sanitized_name = re.sub(r'\W+', '_', sanitized_name)
    return sanitized_name

In [5]:
# Define your list of websites and keyword
websites = ["https://autragency2.com", "https://claimcompass.info", "https://autragency.com"]
keyword = "online casino"

# Tesseract

In [6]:
# Visit websites, search for the keyword, and take screenshots only if the keyword is found
for website in websites:
    try:
        driver.get(website)
        time.sleep(2)  # Give some time for the page to load

        # Check if the keyword is in the page source
        page_source = driver.page_source.lower()
        keyword_found_in_text = keyword.lower() in page_source

        # If the keyword is not in the text, use OCR to check images
        if not keyword_found_in_text:
            temp_screenshot_filename = 'temp_screenshot.png'  # Define the filename
            driver.save_screenshot(temp_screenshot_filename)

            # Perform OCR on the screenshot
            img = Image.open(temp_screenshot_filename)
            text_in_image = pytesseract.image_to_string(img)

            # Remove the temporary screenshot
            os.remove(temp_screenshot_filename)

            # Check if the keyword is in the OCR text
            if keyword.lower() in text_in_image.lower():
                # Save the screenshot with the website name if the keyword is found in the image
                screenshot_filename = f"{sanitize_filename(website)}.png"
                img.save(screenshot_filename)
                print(f"Keyword '{keyword}' found in an image on {website}. Saved screenshot as {screenshot_filename}")
            else:
                print(f"Keyword '{keyword}' not found in text or images on {website}.")
        else:
            # Save the screenshot with the website name if the keyword is found in the text
            screenshot_filename = f"{sanitize_filename(website)}.png"
            driver.save_screenshot(screenshot_filename)
            print(f"Keyword '{keyword}' found in text on {website}. Saved screenshot as {screenshot_filename}")
    except WebDriverException as e:
        if "ERR_NAME_NOT_RESOLVED" in str(e):
            print(f"The website {website} could not be reached (Name not resolved). Skipping...")
        else:
            print(f"An error occurred while trying to access {website}: {e}")
        continue  # Skip to the next website


driver.quit()

Keyword 'online casino' not found in text or images on https://autragency2.com.
Keyword 'online casino' found in text on https://claimcompass.info. Saved screenshot as claimcompass_info.png
The website https://autragency.com could not be reached (Name not resolved). Skipping...
